# Exercise: Use a foundation model to build a spam email classifier

A foundation model serves as a fundamental building block for potentially endless applications. One application we will explore in this exercise is the development of a spam email classifier using only the prompt. By leveraging the capabilities of a foundation model, this project aims to accurately identify and filter out unwanted and potentially harmful emails, enhancing user experience and security.

## Steps

1. Identify and gather relevant data
2. Build and evaluate the spam email classifier
3. Build an improved classifier?

## Step 1: Identify and gather relevant data

To train and test the spam email classifier, you will need a dataset of emails that are labeled as spam or not spam. It is important to identify and gather a suitable dataset that represents a wide range of spam and non-spam emails.

In [3]:
!pip install --upgrade datasets huggingface_hub

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/d7/84/0df6c5981f5fc722381662ff8cfbdf8aad64bec875f75d80b55bfef394ce/datasets-3.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface_hub from https://files.pythonhosted.org/packages/61/8c/fbdc0a88a622d9fa54e132d7bf3ee03ec602758658a2db5b339a65be2cfe/huggingface_hub-0.27.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow>=15.0.0 from https://files.pythonhosted.org/packages/9e/4d/a4988e7d82f4fbc797715db4185939a658eeffb07a25bab7262bed1ea076/pyarrow-18.1.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for requests>=2.32.2 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for tqdm>=4.66.3 from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762ae

In [4]:
# Find a spam dataset at https://huggingface.co/datasets and load it using the datasets library 
from datasets import load_dataset

In [ ]:
dataset = load_dataset("sms_spam", split=["train"])[0]

In [ ]:
for entry in dataset.select(range(3)):
    sms = entry["sms"]
    label = entry["label"]
    print(f"SMS: {sms}\nLabel: {label}\n")

In [ ]:
# Conveniennt dictionaries to convert between labels and ids
id2label = {0: "ham", 1: "spam"}
label2id = {"ham": 0, "spam": 1}    

for entry in dataset.select(range(3)):
    sms = entry["sms"]
    label = entry["label"]
    print(f"SMS: {sms}\nLabel: {id2label[label]}\n")

Step 2: Build and evaluate the spam email classifier
Using the foundation model and the prepared dataset, you can create a spam email classifier.

Let's write a prompt that will ask the model to classify 15 message as either "spam" or "not spam". For easier parsing, we can ask the LLM to respond in JSON

In [ ]:
# Let's start with this helper function that will help us format sms messages
# for the LLM.
def get_sms_messages_string(dataset, item_numbers, include_labels=False):
    sms_messages_string = ""
    for item_number, entry in zip(item_numbers, dataset.select(item_numbers)):
        sms = entry["sms"]
        label_id = entry["label"]

        if include_labels:
            sms_messages_string += (
                f"{item_number} (label={id2label[label_id]}) -> {sms}\n"
            )
        else:
            sms_messages_string += f"{item_number} -> {sms}\n"

    return sms_messages_string


print(get_sms_messages_string(dataset, range(3), include_labels=True))

In [ ]:
# Replace <MASK> with your code

# Get a few messages and format them as a string
sms_messages_string = get_sms_messages_string(dataset, range(7, 15))

# Construct a query to send to the LLM including the sms messages.
# Ask it to respond in JSON format.
query = f"""
{sms_messages_string}
---
Classify the messages above as SPAM or NOT SPAM. Respond in JSON format.
Use the following format : {{"0": "NOT SPAM", "1": "SPAM"}}
"""

print(query)



In [ ]:
# response = !openai api completions.create -d "$query" -m davinci-codex -o json
response = {
  "7": "NOT SPAM",
  "8": "SPAM",
  "9": "SPAM",
  "10": "NOT SPAM",
  "11": "SPAM",
  "12": "SPAM",
  "13": "NOT SPAM",
  "14": "NOT SPAM"
}
